In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import fftpack
import matplotlib.pyplot as plt

absent_dir = '/Users/kushalprakash/Documents/Uni/23-24/AIS+ML/WithoutBaby_train/'

# Fetch all the files for baby absent data which will be used for training

file_paths_absent = []

all_files = os.listdir(absent_dir)    
csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
for i in csv_files:
    file_path = absent_dir + i
    file_paths_absent.append(file_path)
file_paths_absent

In [ ]:
# Initialize an empty list to store the DataFrames
dfs = []

# Loop through the file paths and append each DataFrame to the list
for file in file_paths_absent:
    df = pd.read_csv(file, header=None, index_col=False)
    dfs.append(df)

# Concatenate all the DataFrames in the list
combined_df_absent = pd.concat(dfs, ignore_index=True)
train_absent_len = len(combined_df_absent)
print(train_absent_len)
combined_df_absent

In [ ]:
print(len(combined_df_absent))

# Get the shape of the DataFrame
shape = combined_df_absent.shape

# Number of rows
num_rows = shape[0]

# Number of columns
num_columns = shape[1]

# Print the number of rows and columns
print("Number of rows baby absent:", num_rows)
print("Number of columns baby absent:", num_columns)

In [ ]:
# Skip header data and consider only adc data for fft calculation
adc_data_selected_absent = combined_df_absent.iloc[:, 16:].mean(axis=1)
adc_data_selected_absent

In [ ]:
# Assuming `adc_data` is your pandas Series with ADC data
adc_array = adc_data_selected_absent.to_numpy()  # Convert the pandas Series to a numpy array

# Choose a window function - Hanning window in this case
window = np.hanning(len(adc_array))

# Apply the window function to your data
windowed_adc_data = adc_array * window

# Perform FFT on the windowed data
fft_result = np.fft.fft(windowed_adc_data)

# Frequency bins (assuming you know the sampling rate)
sampling_rate = 16034  # Example: 1000 Hz, replace with your actual sampling rate
n = len(adc_array)
freq = np.fft.fftfreq(n, d=1/sampling_rate)
# Calculate the magnitude and phase of the FFT result
magnitude = np.abs(fft_result)
phase = np.angle(fft_result)

# Create a DataFrame
fft_df = pd.DataFrame({
    'Frequency': freq,
    'FFT Magnitude': magnitude,
    'Phase': phase
})


fft_df

In [ ]:
plt.figure(figsize=(10, 6))  # Set the figure size for better readability
plt.plot(fft_df['Frequency'], fft_df['FFT Magnitude'])  # Plot positive frequency vs magnitude
plt.title('Magnitude Spectrum (Positive Frequencies)')  # Title of the plot
plt.xlabel('Frequency (Hz)')  # Label for the x-axis
plt.ylabel('Magnitude')  # Label for the y-axis
plt.grid(True)  # Show grid for better readability
plt.show()  # Display the plot

In [ ]:
numpy_array = fft_df.to_numpy()
print(os.getcwd())
# Save the array to a file
np.save((absent_dir+'withoutbaby_test.npy'), numpy_array)